In [2]:
import os
import aqi
import pandas as pd
import csv
import numpy as np
os.environ["AQIPY_TOKEN"] = "4924a3c7c66a638cd6d7e12753abb1fb21dc792a"

In [3]:
from waqi_python import client as base
client = base.WaqiClient()

In [4]:
global df_AQI
df_AQI = pd.DataFrame(
{ "PhageID" : [],
"lat" : [],
"long" : [],
"AQI" : [],
"PM25" : []})

In [5]:
#take lat long and phage --> AQI, Pollutant
def addAQI(phageID, lat,long):
    client = base.WaqiClient()
    location = client.get_station_by_latlng(lat, long)
    locationAQI = location.aqi
    PM25 = aqi.to_cc(aqi.POLLUTANT_PM25, locationAQI, algo=aqi.ALGO_EPA)
    PM10 = aqi.to_cc(aqi.POLLUTANT_PM10, locationAQI, algo=aqi.ALGO_EPA)
    CO_8H = aqi.to_cc(aqi.POLLUTANT_CO_8H, locationAQI, algo=aqi.ALGO_EPA)
    NO2_1H = aqi.to_cc(aqi.POLLUTANT_NO2_1H, locationAQI, algo=aqi.ALGO_EPA)
    SO3_1H = aqi.to_cc(aqi.POLLUTANT_SO2_1H, locationAQI, algo=aqi.ALGO_EPA )
    temp_df = pd.DataFrame({'PhageID':[phageID],'lat':[lat], 'long':[long], 'AQI':[locationAQI], 'PM25':[PM25], 
                            'PM10':[PM10], 'CO_8H':[CO_8H], 'NO2_1H':[NO2_1H],'SO3_1H':[SO3_1H]})
    return df_AQI.append(temp_df, ignore_index='False')


In [6]:
phage_df = pd.read_csv('SequencedPhages.csv')
phage_df = phage_df[(phage_df['Found Country'] == 'USA') | (phage_df['Found Country'] == 'United States')]
phage_df = phage_df.dropna(subset = ['Found GPS Lat', 'Found GPS Long'])[['Phage Name', 'Found GPS Lat', 'Found GPS Long']]
phage_df = phage_df.reset_index(drop=True)

In [7]:
phage_df

,Phage Name,Found GPS Lat,Found GPS Long
0,244,40.444259 N,79.953208 W
1,Aaronocolus,33.213 N,97.148 W
2,Abba,34.067996 N,118.450765 W
3,AbbeyMikolon,41.244625 N,75.88789 W
4,AbbysRanger,37.39757 N,85.01984 W
...,...,...,...
2891,ZooBear,33.19747 N,97.132884 W
2892,Zorro,40.957018 N,76.877916 W
2893,Zuko,38.7725 N,90.5564 W
2894,Zulu,39.172716 N,119.719321 W


In [27]:
for ind in range(2874, len(phage_df.index)): #phage_df.index:
    if (25 <= float(phage_df["Found GPS Lat"][ind][:-2]) <= 50) & (-125 <= float(phage_df["Found GPS Long"][ind][:-2])*-1 <= -65):
        df_AQI = addAQI(phage_df["Phage Name"][ind],
                      float(phage_df["Found GPS Lat"][ind][:-2]),
                      float(phage_df["Found GPS Long"][ind][:-2])*-1)
    print(ind)

2874
2875
2876
2877
2878
2879
2880
2881
2882
2883
2884
2885
2886
2887
2888
2889
2890
2891
2892
2893
2894
2895


In [28]:
df_AQI

,PhageID,lat,long,AQI,PM25,PM10,CO_8H,NO2_1H,SO3_1H
0,Zenon,44.555822,-67.863971,33.0,7.9,35,2.9,34,23
1,Zephyr,38.645911,-90.314604,40.0,9.6,43,3.5,42,28
2,Zepp,30.551370,-87.215760,25.0,6.0,27,2.2,26,17
3,Zerg,40.009000,-105.268500,31.0,7.4,33,2.7,32,21
4,Zeta1847,40.840000,-96.650000,21.0,5.0,22,1.8,22,14
...,...,...,...,...,...,...,...,...,...
2892,ZooBear,33.197470,-97.132884,55.0,14.0,63,4.9,57,39
2893,Zorro,40.957018,-76.877916,30.0,7.2,32,2.6,31,21
2894,Zuko,38.772500,-90.556400,37.0,8.8,39,3.2,39,25
2895,Zulu,39.172716,-119.719321,30.0,7.2,32,2.6,31,21


In [29]:
df_AQI.to_csv('df_AQI.csv')